In [1]:
!pip install ibmos2spark

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20190828175753-0000
KERNEL_ID = 027b51da-7240-4a5f-a670-c35a048fb210


In [2]:
# Configuration
# Each country has a tsv file with dates and exchange rates against
# the U.S. dollar

files = ['Australia.tsv', 'Brazil.tsv', 'Canada.tsv', 'China.tsv', 
         'Denmark.tsv', 'EU.tsv', 'Hong_Kong.tsv', 'India.tsv', 
         'Japan.tsv', 'Malaysia.tsv', 'New_Zealand.tsv', 'Norway.tsv',
         'Singapore.tsv', 'South_Africa.tsv', 'South_Korea.tsv',
         'Sri_Lanka.tsv', 'Sweden.tsv', 'Switzerland.tsv', 'Taiwan.tsv',
         'Thailand.tsv', 'United_Kingdom.tsv','Venezuela.tsv']

In [3]:
import ibmos2spark
from pyspark.sql.types import *

# Configuration
# Prepare to access IBM Object Storage

# @hidden_cell
credentials = {
    'service_id': 'iam-ServiceId-e4b1b62b-e030-4b68-a71f-a12e44f23c33',
    'api_key': '9-RaWWhWwA-jm5BkfQ1pxtj-J5xvpjYKMrSgMk9vQxX1',
    'endpoint': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'iam_service_endpoint': 'https://iam.ng.bluemix.net/oidc/token',
    'bucket': 'forex-donotdelete-pr-ksiby6mz5os7oj'
}

# configuration_name = 'os_forex'
configuration_name = ''
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

print(type(cos))

bucket_name = 'forex-donotdelete-pr-ksiby6mz5os7oj'

<class 'ibmos2spark.osconfig.CloudObjectStorage'>


In [4]:
# 'Date' formats come in as a string separated by '-' with:
#  1 or 2 day-digits
#  three letter month categoricals
#  2 digit year-digits

def reformatMonth(a):
    month_dict = {'Jan':'01',
              'Feb':'02',
              'Mar':'03',
              'Apr':'04',
              'May':'05',
              'Jun':'06',
              'Jul':'07',
              'Aug':'08',
              'Sep':'09',
              'Oct':'10',
              'Nov':'11',
              'Dec':'12'
             }
    try: 
        x = re.search(r"([0-9]+-)(.*)(-[0-9][0-9])", a)
        if len(x.groups()[0])==3:
            return ''.join([x.groups()[0], month_dict[x.groups()[1]], x.groups()[2]])
        else:
            return ''.join(['0',x.groups()[0], month_dict[x.groups()[1]], x.groups()[2]])
    except:
        return None

In [5]:
import re
from datetime import datetime 
from pyspark.sql.types import StructType, StructField, StringType, FloatType, DateType

# Loop through all tsv files 
# Remove rows with incomplete data, such as 'ND'
# Reformat 'Date' fields
# Set correct data types for all fields 

schema = StructType([StructField("Date", DateType(), True),
                     StructField("Rate", FloatType(), True)
                    ])

df_all = None
countries = []

for object_name in files:
    data_url = cos.url(object_name, bucket_name)
    df = spark.read.format("csv").option("header", True).option("delimiter", "\t").option("nullValue",'ND').load(data_url) #.schema(forexSchema)
    df2 = df.where(df.Rate.isNotNull())
    print(object_name + " contains " + str(df2.count()) + " rows")
    tmp = re.search(r"(.*)\.tsv", object_name)
    country_name = tmp.groups()[0]
    countries.append(country_name)
    rdd = df2.rdd.map(lambda r: (datetime.strptime(reformatMonth(r[0]), '%d-%m-%y'),float(r[1])))
    df_new = sqlContext.createDataFrame(rdd, schema)
    df_new = df_new.withColumnRenamed('Date', 'Date2')
    df_new = df_new.withColumnRenamed('Rate', country_name)


    if df_all == None:
        df_all = df_new.withColumnRenamed('Date2', 'Date')
        print("Creating dataframe for country " + country_name)
    else:
        # df_all = df_all.union(df_new)
        df_all = df_all.join(df_new, df_all.Date == df_new.Date2)
        df_all = df_all.drop('Date2')
        print("Joining to dataframe for country " + country_name)
df_all = df_all.sort('Date', ascending=True)
df_all.take(5)

Australia.tsv contains 4926 rows
Creating dataframe for country Australia
Brazil.tsv contains 4927 rows
Joining to dataframe for country Brazil
Canada.tsv contains 4927 rows
Joining to dataframe for country Canada
China.tsv contains 4923 rows
Joining to dataframe for country China
Denmark.tsv contains 4927 rows
Joining to dataframe for country Denmark
EU.tsv contains 4927 rows
Joining to dataframe for country EU
Hong_Kong.tsv contains 4927 rows
Joining to dataframe for country Hong_Kong
India.tsv contains 4926 rows
Joining to dataframe for country India
Japan.tsv contains 4927 rows
Joining to dataframe for country Japan
Malaysia.tsv contains 4927 rows
Joining to dataframe for country Malaysia
New_Zealand.tsv contains 4927 rows
Joining to dataframe for country New_Zealand
Norway.tsv contains 4927 rows
Joining to dataframe for country Norway
Singapore.tsv contains 4927 rows
Joining to dataframe for country Singapore
South_Africa.tsv contains 4926 rows
Joining to dataframe for country Sou

[Row(Date=datetime.date(2000, 1, 4), Australia=0.6561999917030334, Brazil=1.840499997138977, Canada=1.451799988746643, China=8.279899597167969, Denmark=7.2179999351501465, EU=1.030900001525879, Hong_Kong=7.777500152587891, India=43.54999923706055, Japan=103.08999633789062, Malaysia=3.799999952316284, New_Zealand=0.5198000073432922, Norway=7.934000015258789, Singapore=1.653499960899353, South_Africa=6.085000038146973, South_Korea=1122.5, Sri_Lanka=72.6500015258789, Sweden=8.359999656677246, Switzerland=1.55649995803833, Taiwan=30.600000381469727, Thailand=37.130001068115234, United_Kingdom=1.6369999647140503, Venezuela=0.6503000259399414),
 Row(Date=datetime.date(2000, 1, 5), Australia=0.6549999713897705, Brazil=1.8559999465942383, Canada=1.451799988746643, China=8.279800415039062, Denmark=7.208000183105469, EU=1.0334999561309814, Hong_Kong=7.7779998779296875, India=43.54999923706055, Japan=103.7699966430664, Malaysia=3.799999952316284, New_Zealand=0.5170999765396118, Norway=7.934999942

In [9]:
data_url = cos.url("etl.parquet", bucket_name)
df_all.write.parquet(data_url)
